# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

names(rrValueWtd) <- names(valueWtd)
names(rrEqualWtd) <- names(equalWtd)

In [4]:
minValWtd <- apply(rrValueWtd, 2, min)
minValWtd <- t(data.frame(minValWtd))
print(minValWtd)
minValWtd <- rbind(minValWtd, row.names(rrValueWtd))
print(minValWtd)
colnames(minValWtd) <- c("I", "MIN")
print(minValWtd)

minEqlWtd <- t(apply(rrEqualWtd, 2, min))

maxValWtd <- apply(rrValueWtd, 2, max)
maxEqlWtd <- apply(rrEqualWtd, 2, max)

sdValWtd <- apply(rrValueWtd, 2, sd)
sdEqlWtd <- apply(rrEqualWtd, 2, sd)

statsValDf <- merge.data.frame(merge.data.frame(minValWtd, maxValWtd), sdValWtd)
statsEqlDf <- merge.data.frame(merge.data.frame(minEqlWtd, maxEqlWtd), sdEqlWtd)

print("MKT CAP WT")
print(statsValDf)

print("EQ WT")
print(statsEqlDf)

                AERO      AGRIC      AUTOS      BANKS       BEER      BLDMT
minValWtd -0.2532453 -0.4074237 -0.7431826 -0.7484332 -0.2183618 -0.5767641
               BOOKS      BOXES      BUSSV     CHEMS     CHIPS      CLTHS
minValWtd -0.7936408 -0.4499764 -0.4315746 -0.360948 -0.765727 -0.3782547
               CNSTR       COAL   DRUGS      ELCEQ      FABPR       FIN
minValWtd -0.5927093 -0.9631609 -0.2876 -0.3043374 -0.6453013 -0.610047
                FOOD       FUN       GOLD       GUNS     HARDW       HLTH
minValWtd -0.1204453 -0.699623 -0.7739683 -0.3086047 -0.742128 -0.3694879
               HSHLD     INSUR      LABEQ       MACH      MEALS      MEDEQ
minValWtd -0.2153368 -0.548031 -0.5442701 -0.3163298 -0.2264927 -0.2991468
               MINES        OIL      OTHER      PAPER      PERSV      RLEST
minValWtd -0.5143283 -0.3595542 -0.5805491 -0.5082174 -0.3867057 -0.7545607
               RTAIL      RUBBR      SHIPS      SMOKE      SODA      SOFTW
minValWtd -0.2353749 -0.4900953

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [2] not equal to array extent


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)